# Import Libraries



In [2]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd
!pip install pandas

### IGNORED, DISMISSED: How the NYPD Neglects 311 Complaints about Driver Misconduct

1. Go to the New Data Portal: https://opendata.cityofnewyork.us/
2. Find each dataset. 311 requests and police precinct
3. Add a data notes section to your github readme. This should include information about the source of your data, where it came from, how often it is updated, and the period of data it contains (e.g. 2010-present or 2012 to 2019)
4. Lastly, test that this import works. This may take a while as the dataset is large.

In [3]:
# query NYC 311 for driver misconduct
# value list
complaint_types_dm = ('Blocked Bike Lane', 'Blocked Crosswalk', 'Blocked Hydrant', 'Blocked Sidewalk', 'Chronic Speeding', 'Chronic Stoplight Violation','Detached Trailer', 'Double Parked Blocking Traffic',
       'Double Parked Blocking Vehicle', 'Drag Racing',
       'Overnight Commercial Storage', 'Parking Permit Improper Use',
       'Posted Parking Sign Violation', 'Truck Route Violation',
       'Unauthorized Bus Layover', 'Abandoned Vehicle With License Plate' )
query = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv?$query=" +  requests.utils.quote(f"SELECT * WHERE descriptor IN {complaint_types_dm}  LIMIT 3000000")
dm  = pd.read_csv(query)

<ipython-input-3-c56bcde3c9d2>:9: DtypeWarning: Columns (15,18,20,31,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  dm  = pd.read_csv(query)


In [ ]:
dm.shape


(2194967, 41)

In [ ]:
dm.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,60886001,2024-04-17T01:51:00.000,NaN,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,Street/Sidewalk,11367.0,67-45 KISSENA BOULEVARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.734313,-73.814760,"\n, \n(40.73431333002716, -73.81476009341641)"
1,60890122,2024-04-17T01:50:05.000,NaN,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,Street/Sidewalk,11365.0,162-05 71 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.730433,-73.806956,"\n, \n(40.730433143081356, -73.80695571889713)"
2,60883280,2024-04-17T01:46:21.000,NaN,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,11219.0,40 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.644535,-73.994267,"\n, \n(40.64453528440371, -73.99426690452185)"
3,60886002,2024-04-17T01:35:34.000,NaN,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,Street/Sidewalk,11385.0,66-32 MYRTLE AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.701412,-73.887130,"\n, \n(40.701412217400446, -73.88713005570222)"
4,60888756,2024-04-17T01:23:16.000,NaN,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11416.0,86-11 95 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.685004,-73.854880,"\n, \n(40.68500350635057, -73.85488009684605)"


In [4]:
dm.columns #investigate closed and open date

Index(['unique_key', 'created_date', 'closed_date', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'location_type', 'incident_zip',
       'incident_address', 'street_name', 'cross_street_1', 'cross_street_2',
       'intersection_street_1', 'intersection_street_2', 'address_type',
       'city', 'landmark', 'facility_type', 'status', 'due_date',
       'resolution_description', 'resolution_action_updated_date',
       'community_board', 'bbl', 'borough', 'x_coordinate_state_plane',
       'y_coordinate_state_plane', 'open_data_channel_type',
       'park_facility_name', 'park_borough', 'vehicle_type',
       'taxi_company_borough', 'taxi_pick_up_location', 'bridge_highway_name',
       'bridge_highway_direction', 'road_ramp', 'bridge_highway_segment',
       'latitude', 'longitude', 'location'],
      dtype='object')

In [26]:
dm['created_date'] = pd.to_datetime(dm['created_date'])
dm['closed_date'] = pd.to_datetime(dm['closed_date'])
dm['duration'] = dm['closed_date'] - dm['created_date']

filtered_dm = dm[dm['duration'] < pd.Timedelta(minutes=5)]

In [17]:
dm['duration'] = pd.to_timedelta(dm['duration'])

In [18]:
dm.sample()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location,duration,<5_min_closure
983170,52271415,2021-10-22 08:45:41,2021-10-22 09:16:32,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,Street/Sidewalk,11238.0,805 WASHINGTON AVENUE,...,NaN,NaN,NaN,NaN,NaN,40.672257,-73.962678,"\n, \n(40.67225714352234, -73.96267793223528)",0 days 00:00:00.000000030,False


In [11]:
dm.sample(3)

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location,duration
1902008,33582086,2016-06-13 11:09:26,2016-06-14 00:33:53,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,10013.0,124 AVENUE OF THE AMERICAS,...,NaN,NaN,NaN,NaN,NaN,NaN,40.724345,-74.004492,"\n, \n(40.72434506017994, -74.00449167298369)",804.450000
1624622,39989222,2018-08-12 10:46:14,2018-08-13 05:20:15,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,Street/Sidewalk,11213.0,1328 PARK PLACE,...,NaN,NaN,NaN,NaN,NaN,NaN,40.672480,-73.933045,"\n, \n(40.67247976509134, -73.93304499904563)",1114.016667
2136242,23184470,2012-05-07 09:57:33,2012-05-07 12:22:38,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11211.0,321 GRAHAM AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,40.713904,-73.944294,"\n, \n(40.71390432891684, -73.94429393967539)",145.083333


In [13]:
dm['<5_min_closure'] = 'filtered_dm'

In [14]:
dm.sample()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location,duration,<5_min_closure
2109828,25300288,2013-04-04 22:42:07,2013-04-04 23:17:28,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,Street/Sidewalk,10001.0,350 WEST 26 STREET,...,NaN,NaN,NaN,NaN,NaN,40.747928,-73.998834,"\n, \n(40.74792806001232, -73.99883427794886)",35.35,filtered_dm


In [19]:
dm['<5_min_closure'] = 'False'

In [20]:
dm.loc[dm['duration'] < pd.Timedelta(minutes=5), '<5_min_closure'] = 'True'

In [21]:
dm.sample()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location,duration,<5_min_closure
986894,52220378,2021-10-18 18:14:57,2021-10-18 19:02:18,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,Street/Sidewalk,11226.0,79 WOODRUFF AVENUE,...,NaN,NaN,NaN,NaN,NaN,40.653516,-73.963197,"\n, \n(40.65351595922442, -73.96319657207762)",0 days 00:00:00.000000047,True


In [ ]:
dm.columns

Index(['unique_key', 'created_date', 'closed_date', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'location_type', 'incident_zip',
       'incident_address', 'street_name', 'cross_street_1', 'cross_street_2',
       'intersection_street_1', 'intersection_street_2', 'address_type',
       'city', 'landmark', 'facility_type', 'status', 'due_date',
       'resolution_description', 'resolution_action_updated_date',
       'community_board', 'bbl', 'borough', 'x_coordinate_state_plane',
       'y_coordinate_state_plane', 'open_data_channel_type',
       'park_facility_name', 'park_borough', 'vehicle_type',
       'taxi_company_borough', 'taxi_pick_up_location', 'bridge_highway_name',
       'bridge_highway_direction', 'road_ramp', 'bridge_highway_segment',
       'latitude', 'longitude', 'location', 'duration', '<5_min_closure'],
      dtype='object')

In [ ]:
dm.head(4)

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location,duration,<5_min_closure
0,60938395,2024-04-22 01:47:11,NaT,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,Street/Sidewalk,11235.0,302 OCEAN VIEW AVENUE,...,NaN,NaN,NaN,NaN,NaN,40.579012,-73.964786,"\n, \n(40.5790117976162, -73.96478559193572)",NaT,False
1,60940393,2024-04-22 01:39:06,NaT,NYPD,New York City Police Department,Illegal Parking,Double Parked Blocking Traffic,Street/Sidewalk,11212.0,612 CHESTER STREET,...,NaN,NaN,NaN,NaN,NaN,40.656565,-73.908366,"\n, \n(40.65656499095104, -73.90836564187046)",NaT,False
2,60941348,2024-04-22 01:38:46,NaT,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11373.0,92-17 54 AVENUE,...,NaN,NaN,NaN,NaN,NaN,40.738106,-73.870321,"\n, \n(40.738105700370596, -73.87032076493618)",NaT,False
3,60936410,2024-04-22 01:38:28,NaT,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11214.0,2069 85 STREET,...,NaN,NaN,NaN,NaN,NaN,40.604371,-73.996226,"\n, \n(40.60437084599537, -73.99622585195348)",NaT,False


In [ ]:
filtered_dm.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location,duration
25,60889182,2024-04-17 00:42:40,2024-04-17 00:45:07,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,10469.0,2903 HERING AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,40.867942,-73.853758,"\n, \n(40.86794172611595, -73.85375775998308)",0 days 00:02:27
126,60886438,2024-04-16 22:24:50,2024-04-16 22:29:47,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,11205.0,192 SPENCER STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,40.692467,-73.954766,"\n, \n(40.6924668113589, -73.95476577524396)",0 days 00:04:57
194,60886434,2024-04-16 21:04:40,2024-04-16 21:07:25,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11106.0,35-17 BROADWAY,...,NaN,NaN,NaN,NaN,NaN,NaN,40.760382,-73.921979,"\n, \n(40.76038200653661, -73.92197947120543)",0 days 00:02:45
281,60883487,2024-04-16 19:30:20,2024-04-16 19:33:37,NYPD,New York City Police Department,Illegal Parking,Blocked Crosswalk,Street/Sidewalk,10309.0,705 RAMONA AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,40.535544,-74.209268,"\n, \n(40.535543601209085, -74.20926768832949)",0 days 00:03:17
341,60887841,2024-04-16 18:43:45,2024-04-16 18:46:04,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,11220.0,301 65 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,40.639129,-74.023219,"\n, \n(40.63912860071856, -74.02321877705549)",0 days 00:02:19


In [ ]:
print(filtered_dm)

         unique_key        created_date         closed_date agency  \
25         60889182 2024-04-17 00:42:40 2024-04-17 00:45:07   NYPD   
126        60886438 2024-04-16 22:24:50 2024-04-16 22:29:47   NYPD   
194        60886434 2024-04-16 21:04:40 2024-04-16 21:07:25   NYPD   
281        60883487 2024-04-16 19:30:20 2024-04-16 19:33:37   NYPD   
341        60887841 2024-04-16 18:43:45 2024-04-16 18:46:04   NYPD   
...             ...                 ...                 ...    ...   
2150290    20861721 2011-07-19 17:29:12 2011-07-19 17:34:01   NYPD   
2158389    20203729 2011-04-09 11:24:48 2011-04-09 11:29:15   NYPD   
2173639    18899750 2010-10-15 11:17:04 2010-10-15 11:21:26   NYPD   
2173863    18873335 2010-10-12 23:11:02 2010-10-12 23:15:56   NYPD   
2178823    18450101 2010-08-10 16:58:32 2010-08-10 17:03:01   NYPD   

                             agency_name   complaint_type  \
25       New York City Police Department  Illegal Parking   
126      New York City Police Departm

In [ ]:
# get police precint data

police = gpd.read_file('https://data.cityofnewyork.us/api/geospatial/78dh-3ptz?method=export&format=GeoJSON')

In [ ]:
police.head()

,precinct,shape_area,shape_leng,geometry
0,1,47285006.2592,80093.5344285,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
1,5,18094527.4385,18807.1249114,"MULTIPOLYGON (((-73.98864 40.72293, -73.98869 ..."
2,6,22017946.5474,24875.9642171,"MULTIPOLYGON (((-73.99968 40.73855, -73.99684 ..."
3,7,18366669.928,17287.5444926,"MULTIPOLYGON (((-73.97346 40.71896, -73.97357 ..."
4,9,21395386.2669,19772.5107407,"MULTIPOLYGON (((-73.97161 40.72672, -73.97163 ..."


In [ ]:
police.shape

(77, 4)